In [8]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import openpyxl

# Load the Excel sheets
brixmor = pd.read_excel(r'M:\Market Research\Research Intern\2024\Matching_databases\MatchTenant_Retailstat.xlsx', sheet_name='brixmor_NAP')
retailstat = pd.read_excel(r'M:\Market Research\Research Intern\2024\\Matching_databases\MatchTenant_Retailstat.xlsx', sheet_name='retailstat')

# Create a function to match DBA names using fuzzy matching
def match_dba(row, choices, threshold=70,method=fuzz.token_set_ratio):
    dba_name = row['DBA'].lower().replace(".","").replace("#","").rstrip(' 1234567890')
    shopping_center = row['BU']
    # Filter choices by shopping center
    filtered_choices = choices[choices['BU'] == shopping_center]
    # Get the best match for the DBA name
    if not filtered_choices.empty:
        best_match = process.extractOne(dba_name, filtered_choices['Tenant'], scorer=method)
        # if best_match and best_match[1] >= threshold:
        return pd.Series([best_match[0], best_match[1]])
    return pd.Series([None, None])

# Apply the function to each row in brixmor and store the results
brixmor[['retailstat_dba', 'match_score']] = brixmor.apply(lambda row: match_dba(row, retailstat), axis=1)
brixmor[['retailstat_dba_2', 'match_score_2']] = brixmor.apply(lambda row: match_dba(row, retailstat,method=fuzz.ratio), axis=1)

# Find rows in brixmor that do not have a corresponding match in retailstat
missing_in_retailstat = brixmor[brixmor['retailstat_dba'].isnull()]

# Select relevant columns to display the missing tenants
missing_tenants = missing_in_retailstat[['LeaseID', 'Latitude', 'Longitude', 'DBA', 'CenterName']]

#Merge Ali
brixmorexport=brixmor.merge(retailstat,left_on=['BU','retailstat_dba'],right_on=['BU','Tenant'])

# Save the updated brixmor dataframe with the new columns
brixmorexport.to_excel(r'M:\Market Research\Research Intern\2024\Matching_databases\updated_brixmor_with_matches_NAP.xlsx', index=False)

# Save the missing tenants to a separate file
missing_tenants.to_excel(r'M:\Market Research\Research Intern\2024\Matching_databases\missing_tenants_fuzzy_matching_NAP.xlsx', index=False)

print("The updated brixmor data and the missing tenants have been identified and saved.")

AttributeError: 'float' object has no attribute 'lower'